## Download GSV Pano and depth map for Launceston using streetlevel python library
* streetlevel is a library for downloading panoramas and metadata from street-level imagery services such as Google Street View, Apple Look Around, and several others. To install the package use: _pip install streetlevel_

## Load modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from streetlevel import streetview
import geopandas as gpd
import json
import os
import re
from PIL import Image
from glob import glob
from aiohttp import ClientSession
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Set input and output folders
* A vector of building points
* Output folders for Panorama images and depth maps

In [ ]:
add_points=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Final_Wagga.shp'
# add_points=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Final_Wagga_training_samples.geojson'
output_folder_pano=r'D:\Wagga\GSV\Pano'
output_folder_depth=r'D:\Wagga\GSV\Depth'
# output_folder_pano=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Panos\streetlevel'
# output_folder_depth=r'C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\Depth_decoded\streetlevel'

## Load GNAF points
We use the points data at Wagga Wagga for demonstration:

In [4]:
gdf_points=gpd.read_file(add_points).to_crs('epsg:4326')
gdf_points.head()

,WALL_M,STEPS,USAGE,STOREYS,ASSESSOR,address,Area_1,PMF,Dep_500,Dep_200,Dep_100,Dep_50,Dep_20,Dep_10,Ground_Lev,Floor_Leve,AGE,UFI,ZONE_DESCR,geometry
0,Metal,1,Commercial,1,ROBERT,3 MOORONG STREET,721.960449,3.6338,-9999.0000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,178.856,179.136,> 1960,2128,Light Industrial,POINT (147.34855 -35.10250)
1,Metal,1,Commercial,1,ROBERT,3 MOORONG STREET,314.036351,3.4780,-9999.0000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,178.856,179.136,> 1960,2129,Light Industrial,POINT (147.34797 -35.10244)
2,Metal,1,Commercial,1,ROBERT,UNIT 1 29 MOORONG STREET,1783.805231,3.2973,0.9431,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,179.137,179.417,> 1960,2130,Light Industrial,POINT (147.34698 -35.10443)
3,Metal,1,Commercial,1,ROBERT,UNIT 1 29 MOORONG STREET,346.207850,4.6240,-9999.0000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,178.455,178.735,> 1960,2141,Light Industrial,POINT (147.34792 -35.10482)
4,Metal,1,Commercial,1,ROBERT,11 MOORONG STREET,615.235606,3.4026,-9999.0000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,178.997,179.277,> 1960,2156,Light Industrial,POINT (147.34845 -35.10319)


### View points

In [5]:
# gdf_points.explore(column='USAGE')

In [6]:
step_m=0.28
tolerance = 1e-12  # A small tolerance to handle floating-point precision
gdf_points['Floor_height']=gdf_points['Floor_Leve']-gdf_points['Ground_Lev']
gdf_points['Floor_height'] = gdf_points['Floor_height'].round(4) # Limit 'Floor_height'decimal places
gdf_points['Ground_surveyed']=1
gdf_points.loc[np.abs(gdf_points['Floor_height'] % step_m) < tolerance,'Ground_surveyed']=0

## Test workflow with first point
### Get point coordinates

In [7]:
i=0
lon=gdf_points.geometry.iloc[i].x
lat=gdf_points.geometry.iloc[i].y

### Record ground-truth FFH

In [8]:
ffh_gt=gdf_points.iloc[i]['Floor_height']
ffh_gt

0.28

### Search and download panorama without depth

In [9]:
zoom=5
pano = streetview.find_panorama(lat, lon)
pano

OhWlnnBJMpUtuvp3gak-8Q (-35.10254, 147.34905) [2019-09]

In [10]:
pano.elevation

181.9572601318359

In [11]:
if pano is None:
    print('cannot find pano for location',lat,lon)
else:
    out_file_pano=os.path.join(output_folder_pano,pano.id+'.jpg')
    if os.path.exists(out_file_pano):
        print('pano file exists, skipping...')
    else:
        print('downloading to file ',out_file_pano)
        async with ClientSession() as session:
            await streetview.download_panorama_async(pano,out_file_pano,session,zoom=zoom)

pano file exists, skipping...


### Download depth map using pano id

In [12]:
pano_with_depth = streetview.find_panorama_by_id(pano.id,download_depth=True)
depth_arr=pano_with_depth.depth.data
depth_img = Image.fromarray(depth_arr)
# plt.imshow(depth_arr)

In [13]:
depth_arr.shape

(256, 512)

Allow large image to be loaded:

In [14]:
Image.MAX_IMAGE_PIXELS=None

### Upscale depth map to the same dimension as pano (optional)

In [15]:
# if os.path.exists(out_file_pano):
#     with Image.open(out_file_pano) as img:
#         pano_size=img.size
# resampled_depth = depth_img.resize(pano_size, resample=Image.BILINEAR)

### Save depth map

In [16]:
out_file_depth=os.path.join(output_folder_depth,pano.id+'.tif')
if os.path.exists(out_file_depth):
    print('pano file exists, skipping...')
else:
    # resampled_depth.save(out_file, format='TIFF',compression='tiff_deflate')
    depth_img.save(out_file_depth, format='TIFF',compression='tiff_deflate')

pano file exists, skipping...


## Batch querying pano metadata

In [17]:
gdf_points_meta=gdf_points.copy()
gdf_points_meta['pano_id']=pd.NA
gdf_points_meta['lat_c']=pd.NA
gdf_points_meta['lng_c']=pd.NA
gdf_points_meta['heading']=pd.NA
gdf_points_meta['pitch']=pd.NA
gdf_points_meta['roll']=pd.NA
gdf_points_meta['elevation']=pd.NA
gdf_points_meta['date']=pd.NA


In [18]:
for i in range(len(gdf_points)):
    lon=gdf_points.geometry.iloc[i].x
    lat=gdf_points.geometry.iloc[i].y
    pano = streetview.find_panorama(lat, lon)
    if not pano is None:
        gdf_points_meta.at[i,'pano_id']=pano.id
        gdf_points_meta.at[i,'lat_c']=pano.lat
        gdf_points_meta.at[i,'lng_c']=pano.lon
        gdf_points_meta.at[i,'heading']=pano.heading
        gdf_points_meta.at[i,'pitch']=pano.pitch
        gdf_points_meta.at[i,'roll']=pano.roll
        gdf_points_meta.at[i,'elevation']=pano.elevation
        gdf_points_meta.at[i,'date']=pano.date
    else:
        print('cannot find pano for location',lat,lon)
gdf_points_meta

cannot find pano for location -35.102436197836965 147.34796699492952
cannot find pano for location -35.10443356055539 147.34697850884078
cannot find pano for location -35.10858842407403 147.34709567147794
cannot find pano for location -35.09845219555239 147.36295593094934
cannot find pano for location -35.098545168891796 147.36266799589302
cannot find pano for location -35.09779490274221 147.36319835649755
cannot find pano for location -35.098733597206284 147.36291522630466
cannot find pano for location -35.098584401524484 147.36293080496193
cannot find pano for location -35.098316285127616 147.36298159389014
cannot find pano for location -35.09814132866165 147.36275449592867
cannot find pano for location -35.0984128145912 147.36270827067938
cannot find pano for location -35.09827986946679 147.36271873211524
cannot find pano for location -35.098694472978586 147.36262702083766
cannot find pano for location -35.09818393685235 147.3630127825462
cannot find pano for location -35.0972264150

,WALL_M,STEPS,USAGE,STOREYS,ASSESSOR,address,Area_1,PMF,Dep_500,Dep_200,...,Floor_height,Ground_surveyed,pano_id,lat_c,lng_c,heading,pitch,roll,elevation,date
0,Metal,1,Commercial,1,ROBERT,3 MOORONG STREET,721.960449,3.6338,-9999.0000,-9999.0,...,0.28,0,OhWlnnBJMpUtuvp3gak-8Q,-35.102543,147.349046,0.169123,-0.014912,6.252468,181.95726,2019-09
1,Metal,1,Commercial,1,ROBERT,3 MOORONG STREET,314.036351,3.4780,-9999.0000,-9999.0,...,0.28,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Metal,1,Commercial,1,ROBERT,UNIT 1 29 MOORONG STREET,1783.805231,3.2973,0.9431,-9999.0,...,0.28,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Metal,1,Commercial,1,ROBERT,UNIT 1 29 MOORONG STREET,346.207850,4.6240,-9999.0000,-9999.0,...,0.28,0,BLJvkN33DWrwpEKVa7q-wQ,-35.104924,147.348552,0.161009,0.002666,6.254132,181.54953,2019-09
4,Metal,1,Commercial,1,ROBERT,11 MOORONG STREET,615.235606,3.4026,-9999.0000,-9999.0,...,0.28,0,PoUl3uYBjG4p2dgCVbWo9g,-35.10322,147.348914,0.147912,0.003101,6.249268,181.555573,2019-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459,Brick,1,Commercial,2,ROBERT,9 STURT STREET,617.507956,4.4999,-9999.0000,-9999.0,...,0.28,0,AWYl-KhhMn2a4fCREJqU3A,-35.105554,147.370953,1.226852,-0.015698,6.242893,185.438202,2022-06
3460,Metal,1,Commercial,1,ROBERT,40-42 DOBNEY AVENUE,575.654113,2.3451,-9999.0000,-9999.0,...,0.28,0,lpc-avq_E8VIEf3T7XBVKQ,-35.12141,147.344241,3.642922,0.000996,6.260234,183.763367,2022-06
3461,Metal,1,Commercial,1,ROBERT,5-7 PEARSON STREET,486.717293,1.0711,-9999.0000,-9999.0,...,0.28,0,e517zGz1x21ZdoLIgIiFEA,-35.122648,147.344012,0.128576,0.021735,6.274247,184.399139,2020-11
3462,Metal,1,Industrial,1,ROBERT,9 PEARSON STREET,981.409060,1.7889,-9999.0000,-9999.0,...,0.28,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Save the metadata as a new vector file

In [19]:
gdf_points_meta['date'] = gdf_points_meta['date'].astype('string')
gdf_points_meta['heading'] = gdf_points_meta['heading'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['lat_c'] = gdf_points_meta['lat_c'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['lng_c'] = gdf_points_meta['lng_c'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['pitch'] = gdf_points_meta['pitch'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['roll'] = gdf_points_meta['roll'].replace(pd.NA, np.nan).astype('float')
gdf_points_meta['elevation'] = gdf_points_meta['elevation'].replace(pd.NA, np.nan).astype('float')

C:\Users\lliu\AppData\Local\Temp\ipykernel_21344\3730501706.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_points_meta['heading'] = gdf_points_meta['heading'].replace(pd.NA, np.nan).astype('float')
C:\Users\lliu\AppData\Local\Temp\ipykernel_21344\3730501706.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_points_meta['lat_c'] = gdf_points_meta['lat_c'].replace(pd.NA, np.nan).astype('float')
C:\Users\lliu\AppData\Local\Temp\ipykernel_21344\3730501706.py:4: FutureWarning: Downcasting behavior in `replace` is deprecat

In [20]:
outfile_meta=r'D:\Wagga\GSV\Final_Wagga_meta.geojson'
gdf_points_meta.to_file(outfile_meta, driver="GeoJSON")
# to avoid overwriting
# if not os.path.exists(outfile_meta):
#     gdf_points_meta.to_file(outfile_meta, driver="GeoJSON")

## Batch downloading

In [21]:
pano_ids=gdf_points_meta['pano_id'][gdf_points_meta['pano_id'].notna()]
len(pano_ids)

3357

In [22]:
for pano_id in pano_ids:
    out_file_pano=os.path.join(output_folder_pano,pano_id+'.jpg')
    if os.path.exists(out_file_pano):
        print('pano file exists, skipping...')
    else:
        pano=streetview.find_panorama_by_id(pano_id,download_depth=False)
        if pano is None:
            print('cannot find pano')
        else:
            try:
                print('downloading to file ',out_file_pano)
                async with ClientSession() as session:
                    await streetview.download_panorama_async(pano,out_file_pano,session,zoom=zoom)
            except Exception as e:
                print(e)

    out_file_depth=os.path.join(output_folder_depth,pano_id+'.tif')
    if os.path.exists(out_file_depth):
        print('depth map exists, skipping...')
    else:
        try:
            pano_with_depth = streetview.find_panorama_by_id(pano_id,download_depth=True)
            if pano_with_depth is None:
                print('cannot decode depth map pano ',pano_id)
            else:
                print('decoding depth map')
                depth_arr=pano_with_depth.depth.data
                depth_img = Image.fromarray(depth_arr)
                # resampled_depth.save(out_file, format='TIFF',compression='tiff_deflate')
                depth_img.save(out_file_depth, format='TIFF',compression='tiff_deflate')
        except Exception as e:
            print(e)

pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
invalid literal for int() with base 2: ''
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping...
depth map exists, skipping...
pano file exists, skipping..